# Recurrent Neural Network


### Imports

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# for reproducibility
torch.manual_seed(100)


### 1 Dataset Construction



In [6]:
# Dictionary

sample_sentence = 'hi!hello.'
# 학습할 데이터 sample_sentence에 저장
char_set = list(set(sample_sentence))
# 중복된 값을 제거한 list를 생성해 char_set에 저장
# -> [h,i,!,e,l,o,.]
dic = {c: i for i, c in enumerate(char_set)}
# enumerate함수를 사용해 char_set의 각각의 value값과 index값을 딕셔너리 함수에 넣어준다
# -> {h:0, i:1, !:2, e:3, l:4, o:5, .:6}
print(char_set)

# Parameters

dic_size = len(dic)
# dic의 길이만큼 dic_size에 대입 -> 7
input_size = dic_size
# dic_size로 input_zise 결정 -> 7
hidden_size = dic_size
# dic_size로 hidden_size 결정 -> 7


# Dataset setting

x_batch = []
# 빈 리스트 생성해 x_batch에 저장
y_batch = []
# 빈 리스트 생성해 y_batch에 저장

x_data = [dic[c] for c in sample_sentence[:-1]]
# 마지막 원소(.)을 제외한 값(input)을 위에 만들어 놓은 dic의 value값에 각각 대입
# -> x_data= [0, 1, 2, 0, 3, 4, 4, 5]
x_one_hot = [np.eye(dic_size)[x] for x in x_data]
# embedding 과정 ->
# x_one_hot =
# h = [1 0 0 0 0 0 0]
# i = [0 1 0 0 0 0 0]
# ! = [0 0 1 0 0 0 0]
# h = [1 0 0 0 0 0 0]
# e = [0 0 0 1 0 0 0]
# l = [0 0 0 0 1 0 0]
# l = [0 0 0 0 1 0 0]
# o = [0 0 0 0 0 1 0]
# 이것이 one_hot encoding
y_data = [dic[c] for c in sample_sentence[1:]]
# sample_date의 [1]부터 모든 원소(output)를 위에 만들어 놓은 dic의 value값에 각각 대입
# y_data = [i=1, !=2, h=0, e=3, l=4, l=4, o=5, .=6]

x_batch.append(x_one_hot)
# x_one_hot 결과를 x_batch의 list에 넣어준다
y_batch.append(y_data)
# y_data의 결과를 y_batch의 list에 넣어준다


# To torch tensors
X = torch.FloatTensor(x_batch)
# x_batch의 tensor값을 X에 대입 -> 배치사이즈(N) = 1, seqence size(S) = 8, Embedding size(E) = 7
Y = torch.LongTensor(y_batch)
# y_batch의 tensor값을 Y에 대입 -> 배치사이즈(N) = 1, hiddne size(H) = 8
print(X.shape)
# X의 형태 출력
print(Y.shape)
# Y의 형태 출력

['!', '.', 'i', 'h', 'o', 'e', 'l']
torch.Size([1, 8, 7])
torch.Size([1, 8])


### 2 RNN model
* Input (입력의 형태)
  + Input type: torch.Tensor
  + Input shape: (N x S x E)
    - N: Batch size, S: Sequence length, E: Embedding size
    - 단, **batch_first=True** 일 때
  + 입력값 'hi!hello'
    - (1, 8, 7)
* Hidden (출력의 형태)
  + Hidden type: torch.Tensor
  + 출력값 'i!hello.'
    - (1, 8, 7)

In [7]:
# Model
learning_rate = 0.1
# learning_rate를 0.1로 설정
training_epochs = 60
# epoch값 60 설정
model = nn.RNN(input_size, hidden_size, batch_first=True)
# 앞에 선언한 각각의 값들을 RNN모델에 넣는다. 여기서 batch_first=True로 지정한 이유는 앞의 형태가 (N x S x E)로 되어 있기 때문이다.

In [8]:
# define cost/loss & optimizer

# 여기부터 전에 다뤄왔던 loss구하는 방법과 동일
criterion = nn.CrossEntropyLoss()    # Softmax
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# train
for epoch in range(training_epochs):
  optimizer.zero_grad()
  outputs, _status = model(X)
  loss = criterion(outputs.reshape(-1, dic_size), Y.reshape(-1))
  loss.backward()
  optimizer.step()
  if epoch % 5 == 4:
    result = outputs.data.numpy().argmax(axis=2)
    result_str = ''.join([char_set[c] for c in np.squeeze(result)])
    print('epoch: ',epoch, 'loss: ', loss.item(), 'prediction: ', result_str, 'true Y: ', sample_sentence[1:])


epoch:  4 loss:  1.2104920148849487 prediction:  .!hello. true Y:  i!hello.
epoch:  9 loss:  0.8444290161132812 prediction:  i!hello. true Y:  i!hello.
epoch:  14 loss:  0.7109715938568115 prediction:  i!hello. true Y:  i!hello.
epoch:  19 loss:  0.6631023287773132 prediction:  i!hello. true Y:  i!hello.
epoch:  24 loss:  0.6405735015869141 prediction:  i!hello. true Y:  i!hello.
epoch:  29 loss:  0.6292124390602112 prediction:  i!hello. true Y:  i!hello.
epoch:  34 loss:  0.6235072016716003 prediction:  i!hello. true Y:  i!hello.
epoch:  39 loss:  0.6202727556228638 prediction:  i!hello. true Y:  i!hello.
epoch:  44 loss:  0.6180737018585205 prediction:  i!hello. true Y:  i!hello.
epoch:  49 loss:  0.6165153384208679 prediction:  i!hello. true Y:  i!hello.
epoch:  54 loss:  0.615384578704834 prediction:  i!hello. true Y:  i!hello.
epoch:  59 loss:  0.6144909858703613 prediction:  i!hello. true Y:  i!hello.


### 3 Assignment
### 다음 3개의 문장을 batch data로 활용해 RNN을 학습해보자 (아래의 미완성 코드, 위 실습코드, 실행결과를 참고)
* 'howareyou'
* 'whats up?'
* 'iamgreat.'

In [12]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# for reproducibility
torch.manual_seed(100)

# Dictionary
sample_sentences = ['howareyou', 'whats up?', 'iamgreat.']
char_set = list(set(''.join(sample_sentences)))
# 공백을 기준으로 모든 문자열을 하나로 합쳐 list를 만들어 준다.
dic = {c: i for i, c in enumerate(char_set)}
# 각각의 value와 index값을 딕셔너리에 넣어주고 그 값을 dic변수에 저장
print(char_set)
# char_set이 원하는 형태로 나오고 있는지 확인하기 위해 중간에 print함수를 넣어주었다.

# Parameters
dic_size = len(dic)
input_size = dic_size
hidden_size = dic_size

# Dataset setting
input_batch = []
target_batch = []

for sentence in sample_sentences:
# 각 문장 3개를 반복문을 통해 진행
    x_data = [dic[c] for c in sentence[:-1]]
    x_one_hot = [np.eye(dic_size)[x] for x in x_data]
    y_data = [dic[c] for c in sentence[1:]]
    input_batch.append(x_one_hot)
    target_batch.append(y_data)

# To torch tensors
X = torch.FloatTensor(input_batch)
Y = torch.LongTensor(target_batch)

# Model
learning_rate = 0.05
training_epochs = 500
model = nn.RNN(input_size, hidden_size, batch_first=True)

# define cost/loss & optimizer
criterion = nn.CrossEntropyLoss()    # Softmax
# Cross-entropy loss는 one-hot 인코딩된 벡터와 직접적으로 비교될 필요 없이 정수 형태의 레이블을 기대한다.
# 따라서 y_data는 각 문자에 대한 인덱스만 포함하면 된다
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# train
for epoch in range(training_epochs):
    optimizer.zero_grad()
    outputs, _status = model(X)
    loss = criterion(outputs.reshape(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        result = outputs.data.numpy().argmax(axis=2)
        for sentence in result:
            decoded_sentence = ''.join([char_set[c] for c in sentence])
            print(decoded_sentence)



['s', 'e', 'm', '.', ' ', 'o', 'h', 'a', 'u', 'p', 'i', 'y', 'w', 'g', 't', '?', 'r']
emet s h
eehmee  
eh h spm
owareaou
hats up?
amgreat.
owareaou
hats up?
amgreat.
owareaou
hats up?
amgreat.
owareaou
hats up?
amgreat.
